In [1]:
import lightgbm

In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [5]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
dependent=dataset[['charges']]

In [11]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
param_grid={"num_leaves": [31, 63, 127], "max_depth": [-1, 3, 5], "subsample": [0.8, 1.0], "colsample_bytree": [0.8, 1.0],
            'n_estimators':[50,100,200,500], 'learning_rate':[0.1,0.2,0.03]}
grid=GridSearchCV(LGBMRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(independent,dependent)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000197 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1338, number of used features: 5
[LightGBM] [Info] Start training from score 13270.422260
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

C:\Users\Raji\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Raji\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Raji\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raji\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\Raji\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(

GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.1, 0.2, 0.03],
                         'max_depth': [-1, 3, 5],
                         'n_estimators': [50, 100, 200, 500],
                         'num_leaves': [31, 63, 127], 'subsample': [0.8, 1.0]},
             verbose=3)

In [13]:
re=grid.cv_results_
print("The R_score value for best parameter {}:".format(grid.best_params_))

The R_score value for best parameter {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'num_leaves': 31, 'subsample': 0.8}:


In [15]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.145220,0.028212,0.004053,0.001282,0.8,0.10,-1,50,31,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",0.870905,0.797926,0.874960,0.841016,0.852002,0.847362,0.027647,181
1,0.176585,0.019400,0.002943,0.000792,0.8,0.10,-1,50,31,1.0,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",0.870905,0.797926,0.874960,0.841016,0.852002,0.847362,0.027647,181
2,0.161963,0.005317,0.003140,0.001039,0.8,0.10,-1,50,63,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",0.867932,0.791633,0.875416,0.841270,0.851602,0.845571,0.029512,197
3,0.157249,0.005185,0.002805,0.000922,0.8,0.10,-1,50,63,1.0,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",0.867932,0.791633,0.875416,0.841270,0.851602,0.845571,0.029512,197
4,0.156504,0.003376,0.003167,0.000157,0.8,0.10,-1,50,127,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",0.867932,0.791633,0.875416,0.841270,0.851602,0.845571,0.029512,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0.609495,0.039085,0.004883,0.000925,1.0,0.03,5,500,31,1.0,"{'colsample_bytree': 1.0, 'learning_rate': 0.0...",0.870598,0.792126,0.881981,0.841273,0.851287,0.847453,0.031119,175
428,0.598605,0.022524,0.005122,0.000945,1.0,0.03,5,500,63,0.8,"{'colsample_bytree': 1.0, 'learning_rate': 0.0...",0.870598,0.792126,0.881981,0.841273,0.851287,0.847453,0.031119,175
429,0.605838,0.041250,0.005021,0.001117,1.0,0.03,5,500,63,1.0,"{'colsample_bytree': 1.0, 'learning_rate': 0.0...",0.870598,0.792126,0.881981,0.841273,0.851287,0.847453,0.031119,175
430,0.694667,0.113947,0.004828,0.000600,1.0,0.03,5,500,127,0.8,"{'colsample_bytree': 1.0, 'learning_rate': 0.0...",0.870598,0.792126,0.881981,0.841273,0.851287,0.847453,0.031119,175


In [17]:
pd.set_option('display.max_rows', None)
print(table)

     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.145220      0.028212         0.004053        0.001282   
1         0.176585      0.019400         0.002943        0.000792   
2         0.161963      0.005317         0.003140        0.001039   
3         0.157249      0.005185         0.002805        0.000922   
4         0.156504      0.003376         0.003167        0.000157   
5         0.160794      0.002061         0.003622        0.001428   
6         0.260821      0.025411         0.002518        0.000649   
7         0.231785      0.001498         0.002819        0.000667   
8         0.287104      0.002691         0.002733        0.000877   
9         0.285993      0.001718         0.002731        0.001009   
10        0.294965      0.002184         0.003137        0.001010   
11        0.326401      0.021178         0.003520        0.001279   
12        0.596496      0.037785         0.005601        0.001555   
13        0.571945      0.092235  

In [19]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 34
BMI: 24
Children: 1
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 1


In [21]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[19587.0622067]
